In [1]:
import pandas as pd
db_train = pd.read_csv('peliculas_sentimiento/train.csv')
db_test = pd.read_csv('peliculas_sentimiento/test.csv')
db_test

,text,sentiment
0,"My daughter liked it but I was aghast, that a ...",neg
1,I... No words. No words can describe this. I w...,neg
2,this film is basically a poor take on the old ...,neg
3,"This is a terrible movie, and I'm not even sur...",neg
4,First of all this movie is a piece of reality ...,pos
...,...,...
24995,"For one thing, he produced this movie. It has ...",neg
24996,The title comes from an alteration an adolesce...,pos
24997,Christopher Nolan's first film is a 'no budget...,pos
24998,The story is shortly about the faith-lacking b...,neg


# Preprocesa los datos

In [2]:
def process_opinions(text):
    text = text.lower() # Pone todo a minusculas
    return list(set(text.split())) # Elimina palabras repetidas y divide el texto en palabras

db_train['words'] = db_train['text'].apply(process_opinions)

# Creamos el modelo y lo probamos

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

vectorizer = CountVectorizer()

db_train['words_text'] = db_train['words'].apply(lambda words: ' '.join(words)) # Convierte en texto plano concatenando las palabras con espacios.
X = vectorizer.fit_transform(db_train['words_text']) # Convierte el texto en una matriz de características numéricas.
y = db_train['sentiment']
X_predict = vectorizer.transform(db_test['text'])  # Vectorizamos los datos de prueba
y_predict = db_test['sentiment']  # Etiquetas reales de sentimiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
NB_opinions = MultinomialNB()
NB_opinions.fit(X, y)

def predict_nb(opinion):
    opinion_vector = vectorizer.transform([opinion]) # Convierte la opinión en una representación numérica compatible con el modelo.
    if (NB_opinions.predict_proba(opinion_vector)[0][1] >= 0.5): # Se toma la probabilidad de que la opinión sea positiva (0 - neg, 1 - pos)
        return  f"{NB_opinions.predict_proba(opinion_vector)[0][1]} -> positiva"
    else: return f"{NB_opinions.predict_proba(opinion_vector)[0][1]} -> negativa"

In [14]:
y_pred = NB_opinions.predict(X_predict)
accuracy = accuracy_score(y_predict, y_pred)
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.8101


In [15]:
print(predict_nb("i dont like it its horrible"))

0.10813121584582314 -> negativa
